# Load 🤖 convert DICOM to 3D volume

In [1]:
# !pip download -q "python-gdcm" pydicom pylibjpeg "opencv-python-headless" --dest frozen_packages --prefer-binary
# !pip wheel -q https://github.com/Borda/kaggle_vol-3D-classify/archive/refs/heads/main.zip --wheel-dir frozen_packages --prefer-binary
# !rm frozen_packages/torch-*
# !ls -lh frozen_packages

^C
ERROR: Operation cancelled by user


In [ ]:
# !pip install -qU "python-gdcm" pydicom pylibjpeg kaggle_vol3d_classify --find-links frozen_packages --no-index

In [1]:
%matplotlib inline

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PATH_DATASET = "/mnt/e/Code/Kaggle/RSNA_data"

### Loading & saving DICOM image

In [2]:
import cv2
import pydicom
import torch
from PIL import Image
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
from pydicom.pixel_data_handlers import apply_voi_lut
from kaggle_volclassif.utils import interpolate_volume
from skimage import exposure
    

def convert_volume(dir_path: str, out_dir: str = "train_volumes", size = (224, 224, 224)):
    ls_imgs = glob.glob(os.path.join(dir_path, "*.dcm"))
    ls_imgs = sorted(ls_imgs, key=lambda p: int(os.path.splitext(os.path.basename(p))[0]))

    imgs = []
    for p_img in ls_imgs:
        dicom = pydicom.dcmread(p_img)
        img = apply_voi_lut(dicom.pixel_array, dicom)
        img = cv2.resize(img, size[:2], interpolation=cv2.INTER_LINEAR)
        imgs.append(img.tolist())
    vol = torch.tensor(imgs, dtype=torch.float32)

    vol = (vol - vol.min()) / float(vol.max() - vol.min())
    vol = interpolate_volume(vol, size).numpy()
    
    # https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_adapt_hist_eq_3d.html
    vol = exposure.equalize_adapthist(vol, kernel_size=np.array([64, 64, 64]), clip_limit=0.01)
    # vol = exposure.equalize_hist(vol)
    vol = np.clip(vol * 255, 0, 255).astype(np.uint8)
    
    path_npz = os.path.join(out_dir, f"{os.path.basename(dir_path)}.npz")
    np.savez_compressed(path_npz, vol)

### Process all images 🤖

In [4]:
from pprint import pprint
from joblib import Parallel, delayed
from tqdm.auto import tqdm

! rm -rf train_volumes
! mkdir train_volumes

ls_dirs = [p for p in glob.glob(os.path.join(PATH_DATASET, "train_images", "*")) if os.path.isdir(p)]
print(f"volumes: {len(ls_dirs)}")

_= Parallel(n_jobs=4)(delayed(convert_volume)(p_dir) for p_dir in tqdm(ls_dirs))

! ls -lh train_volumes

volumes: 2019


  0%|          | 0/2019 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Show 🔎 few samples

In [11]:
from ipywidgets import interact, IntSlider
from matplotlib.patches import PathPatch, Rectangle
from matplotlib.path import Path


def _draw_line(ax, coords, clr='g'):
    line = Path(coords, [Path.MOVETO, Path.LINETO])
    pp = PathPatch(line, linewidth=3, edgecolor=clr, facecolor='none')
    ax.add_patch(pp)

def _set_axes_labels(ax, axes_x, axes_y):
    ax.set_xlabel(axes_x)
    ax.set_ylabel(axes_y)
    ax.set_aspect('equal', 'box')

_rec_prop = dict(linewidth=5, facecolor='none')

def show_volume(vol, z, y, x, fig_size=(9, 9)):
    fig, axarr = plt.subplots(nrows=2, ncols=2, figsize=fig_size)
    v_z, v_y, v_x = vol.shape
    axarr[0, 0].imshow(vol[z, :, :], cmap="gray")
    axarr[0, 0].add_patch(Rectangle((-1, -1), v_x, v_y, edgecolor='r', **_rec_prop))
    _draw_line(axarr[0, 0], [(x, 0), (x, v_y)], "g")
    _draw_line(axarr[0, 0], [(0, y), (v_x, y)], "b")
    _set_axes_labels(axarr[0, 0], "X", "Y")
    axarr[0, 1].imshow(vol[:, :, x].T, cmap="gray")
    axarr[0, 1].add_patch(Rectangle((-1, -1), v_z, v_y, edgecolor='g', **_rec_prop))
    _draw_line(axarr[0, 1], [(z, 0), (z, v_y)], "r")
    _draw_line(axarr[0, 1], [(0, y), (v_x, y)], "b")
    _set_axes_labels(axarr[0, 1], "Z", "Y")
    im = axarr[1, 0].imshow(vol[:, y, :], cmap="gray")
    axarr[1, 0].add_patch(Rectangle((-1, -1), v_x, v_z, edgecolor='b', **_rec_prop))
    _draw_line(axarr[1, 0], [(0, z), (v_x, z)], "r")
    _draw_line(axarr[1, 0], [(x, 0), (x, v_y)], "g")
    _set_axes_labels(axarr[1, 0], "X", "Z")
    plt.colorbar(im, ax=axarr[1, 1])
    axarr[1, 1].set_axis_off()
    fig.tight_layout()


def interactive_show(volume):
    vol_shape = volume.shape
    print(vol_shape)
    interact(
        lambda x, y, z: plt.show(show_volume(volume, z, y, x)),
        z=IntSlider(min=0, max=vol_shape[0]-1, step=2, value=int(vol_shape[0] / 2)),
        y=IntSlider(min=0, max=vol_shape[1]-1, step=5, value=int(vol_shape[1] / 2)),
        x=IntSlider(min=0, max=vol_shape[2]-1, step=5, value=int(vol_shape[2] / 2)),
    )

In [12]:
ls_vols = glob.glob(os.path.join("train_volumes", "*.npz"))

vol = np.load(ls_vols[2])['arr_0']

interactive_show(vol)

(224, 224, 224)


interactive(children=(IntSlider(value=112, description='x', max=223, step=5), IntSlider(value=112, description…